In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [11]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [12]:
feed.routes.route_type.iat[0] = 2
feed.routes.route_type.unique()

array([2, 3])

In [14]:
trip_stats = feed.compute_trip_stats()

In [54]:
#dates = ["20100101"]

feed1 = feed.copy()
c = feed1.calendar
c["monday"] = 0
feed1.calendar = c
dates = feed1.get_first_week()[:1] + ["20100101"]
print(dates)

f = feed1.compute_feed_stats(trip_stats, dates=dates, split_route_types=False)
f.T


['20140526']
hello 20140526    num_stops  num_routes  num_trips  num_trip_starts  num_trip_ends  \
0        NaN         NaN        NaN              NaN            NaN   

   peak_num_trips  peak_start_time  peak_end_time  service_distance  \
0             NaN              NaN            NaN               NaN   

   service_duration  service_speed      date  
0               NaN            NaN  20140526  


,0
num_stops,NaN
num_routes,NaN
num_trips,NaN
num_trip_starts,NaN
num_trip_ends,NaN
peak_num_trips,NaN
peak_start_time,NaN
peak_end_time,NaN
service_distance,NaN
service_duration,NaN


In [52]:
f = pd.DataFrame(columns=["a", "b"])
f.loc[:, "a"] = 3
f

,a,b


In [9]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

,hello,goodbye
0,12,13
1,12,13


In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()